In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

import streamlit as st
import joblib

In [0]:
#Librerias para redes neuronales(secuencial)
from tensorflow.keras.models import Sequential
#Capa completamente conectada
from tensorflow.keras.layers import Dense, Normalization
#Optimizador
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

Modelo Prediccion Pases (XGB)

In [0]:
import joblib
import numpy as np


# Variables a traer de la BD de la Operacion

# Variable 1
al_termino_cargar_en_espera_cuadrado_cuadrandose = True   # Completar este valor con lo que se tiene en el registro o fila de lo que traigamos de la Operacion

tiene_camion_cuadrado = al_termino_cargar_en_espera_cuadrado_cuadrandose

# Variable 2 
capacidad_en_volumen_equipo_acarreo_m3 = 129 # Completar este valor con lo que se tiene en el registro o fila de lo que traigamos de la Operacion

# Variable 3
##En porcentaje, se define cual es el factor de llenado en peso, que es manejado mas por el expertise del Operador (Por el momento se mantiene asi)
factor_llenado_peso = 0.9   # Sera constante por el momento

# Variable 4
#En porcentaje, se define cual es el factor de llenado en peso, que es manejado mas por el expertise del Operador (Por el momento se mantiene asi)
factor_llenado_volumen = 0.9  # Sera constante por el momento

#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Cargamos  el modelo y su scaler para normalizacion guardado (entrenado y listo para predecir)
model_xgb = joblib.load('modelo_xgb_6var_N.pkl')
scaler = joblib.load('scaler_modelo_xgb_6var_N.pkl')

# Variables a traer de la BD de la Operacion  (En particular estas variables serviran para que el Modelo haga la prediccion de pases, 6 variables)
capacidad_en_volumen_equipo_carguio_m3 = 27  # Variable 5
capacidad_en_peso_equipo_carguio = 40   # Variable 6
capacidad_en_peso_equipo_acarreo = 229  # Variable 7
tonelaje_camion_antes_cargaestabilizada = 34  # Variable 8
angulo_giro_promedio_pases = 90    # Variable 9
densidad_inicial_poligono_creado_tn_m3 = 1.9    # Variable 10


# Crear un array con los valores de las variables
input_features = np.array([[capacidad_en_volumen_equipo_carguio_m3, capacidad_en_peso_equipo_carguio, capacidad_en_peso_equipo_acarreo, 
                            tonelaje_camion_antes_cargaestabilizada, angulo_giro_promedio_pases, densidad_inicial_poligono_creado_tn_m3]])

# Normalizar los datos de entrada del usuario
input_features_normalized = scaler.transform(input_features)

# Realizar la predicción utilizando el modelo cargado
prediction = model_xgb.predict(input_features_normalized)

# Finalmente la prediccion de pases, se almacena en esta variable
rounded_prediction = np.round(prediction).astype('int64')

rounded_prediction = rounded_prediction + 2
#---------------------------------------------------------------------------------------------------------------------------------------------------------------

# Variables a traer de la BD de la Operacion

# Variable 11
# En producccion, este valor debera ser reemplazado por el tonelaje que acumula el equipo durante la carga, en tiempo real (dato que sale del equipo)
tonelaje_acumulado_anterior_carga = float(capacidad_en_peso_equipo_carguio * (rounded_prediction - 1))  # Variable calculada 

# Variable 12
#En producccion, este valor debera ser reemplazado por el tonelaje que acumula el equipo durante la carga, en tiempo real (dato que sale del equipo)
tonelaje_acumulado_carga = float(capacidad_en_peso_equipo_carguio * (rounded_prediction))   # Variable calculada

# Variable 13
# Variable que se completo de acuerdo al expertise, pero puedo cambiar en las operaciones
payload = 1.05     # Sera constante por el momento

# Variable 14
# Variable que se completo de acuerdo al expertise, pero puedo cambiar en las operaciones
capacidad_llenado_vol = 1.05  # Sera constante por el momento


# Algoritmo Matematico (Logica para Aumentar O No un Pase Adicional)

#Condicional para hallar la variable calculada "cargado_real_cuchara_ton"
if (capacidad_en_peso_equipo_carguio * factor_llenado_peso / densidad_inicial_poligono_creado_tn_m3) > (capacidad_en_volumen_equipo_carguio_m3 * factor_llenado_volumen):
    cargado_real_cuchara_ton = capacidad_en_volumen_equipo_carguio_m3 * densidad_inicial_poligono_creado_tn_m3 * factor_llenado_volumen
else:
    cargado_real_cuchara_ton = capacidad_en_peso_equipo_carguio * factor_llenado_peso


# Variable 15 
pases_depositados = tonelaje_acumulado_carga / cargado_real_cuchara_ton    #Variable calculada


#Condicional para hallar la variable calculada "calculo_promedio_densidad_ton_m3"
if (capacidad_en_volumen_equipo_carguio_m3 == 0 ):
    calculo_promedio_densidad_ton_m3 = densidad_inicial_poligono_creado_tn_m3
else:
    calculo_promedio_densidad_ton_m3 = densidad_inicial_poligono_creado_tn_m3 * (tonelaje_acumulado_carga - tonelaje_acumulado_anterior_carga) / cargado_real_cuchara_ton 

densidad_material_pase_ton_m3 = np.mean([densidad_inicial_poligono_creado_tn_m3, calculo_promedio_densidad_ton_m3, tonelaje_acumulado_carga/(capacidad_en_volumen_equipo_carguio_m3 * pases_depositados)])


#Condicional para hallar la variable calculada "tonelaje_requerido_camion_ton"
if ((capacidad_en_peso_equipo_acarreo * payload) / densidad_material_pase_ton_m3 >  capacidad_en_volumen_equipo_acarreo_m3 * capacidad_llenado_vol ):
    tonelaje_requerido_camion_ton = capacidad_en_volumen_equipo_acarreo_m3 * densidad_material_pase_ton_m3 * capacidad_llenado_vol
else:
    tonelaje_requerido_camion_ton = capacidad_en_peso_equipo_acarreo * payload


# Variable 16 
numero_cucharadas_requeridos = round(tonelaje_requerido_camion_ton / cargado_real_cuchara_ton, 2)  # Variable calculada

# Variable 17 ( es una variable calculada que ayudara a indicar si se Necesita o No un pase adicional)
diferencia = numero_cucharadas_requeridos - pases_depositados 


#Condicional para indicar si se "Necesita o No" 1 pase adicional
if tiene_camion_cuadrado is True:
    if diferencia > 0.8:
        necesita_un_pase_mas = True
    else:
        necesita_un_pase_mas = False
else:
    if diferencia > 0.2:
        necesita_un_pase_mas = True
    else:
        necesita_un_pase_mas = False

# Mostrar el resultado
print("¿Necesita un pase más?:", necesita_un_pase_mas)

In [0]:
rounded_prediction

Modelo Matematico (Necesita 1 pase mas? Si/No)

In [0]:
datos = pd.read_csv('datos_modelo_qwin1.csv', parse_dates=['tiempo_llegada_camion'], date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S.%f%z'))
datos.head(1)

In [0]:
datos = datos[['capacidad_en_volumen_equipo_carguio_m3',
'capacidad_en_peso_equipo_carguio',
'capacidad_en_peso_equipo_acarreo',
'angulo_giro_promedio_pases',
'densidad_inicial_poligono_creado_tn/m3', 
'al_termino_cargar_en_espera_cuadrado_cuadrandose']]

--Tiene Camión Cuadrado-- = 'al_termino_cargar_en_espera_cuadrado_cuadrandose' (O calcular apartir de la columna tiempo_cuadrado y tiempo cargando pala)

--Numero de cucharadas Requeridos--  = prediction  

#--Factor de llenado volumen-- = Cargado Real Cuchara (ton) / (--Densidad Material PASE (ton/m3)-- * 'capacidad_en_volumen_equipo_carguio_m3' )

--Factor de llenado volumen-- = 90%

#--Factor de llenado peso-- = Cargado Real Cuchara (ton) / 'capacidad_en_peso_equipo_carguio'

--Factor de llenado peso-- = 90%


--Cargado Real Cuchara (ton)-- = si('capacidad_en_peso_equipo_carguio' * --Factor de llenado peso-- /'densidad_inicial_poligono_creado_tn/m3' > 'capacidad_en_volumen_equipo_carguio_m3' * --Factor de llenado volumen--
, 'capacidad_en_volumen_equipo_carguio_m3' * 'densidad_inicial_poligono_creado_tn/m3'* --Factor de llenado volumen-- , 'capacidad_en_peso_equipo_carguio' * --Factor de llenado peso--)
                               
                              
--Pases Depositados-- =  --Tonelaje Acumulado Carga-- / --Cargado Real Cuchara (ton)--

--Necesita Un Pase Más-- = si(--Tiene Camión Cuadrado-- = TRUE, SI (--Numero de cucharadas Requeridos-- - --Pases Depositados-- > 0.8 , TRUE, FALSE), 
                          SI(--Numero de cucharadas Requeridos-- - Pases Depositados > 0.2 , TRUE, FALSE))

In [0]:
# Cambiar el nombre de la columna 'antiguo_nombre' a 'nuevo_nombre'
datos.rename(columns={'al_termino_cargar_en_espera_cuadrado_cuadrandose': 'tiene_camion_cuadrado'}, inplace=True)

In [0]:
# Variables

tiene_camion_cuadrado = datos['tiene_camion_cuadrado'].iloc[0]

capacidad_en_volumen_equipo_acarreo_m3 = datos['capacidad_en_volumen_equipo_acarreo_m3'].iloc[0]


##En porcentaje, se define cual es el factor de llenado en peso, que es manejado mas por el expertise del Operador (Por el momento se mantiene asi)
factor_llenado_peso = 0.9

#En porcentaje, se define cual es el factor de llenado en peso, que es manejado mas por el expertise del Operador (Por el momento se mantiene asi)
factor_llenado_volumen = 0.9


# En producccion, este valor debera ser reemplazado por el tonelaje que acumula el equipo durante la carga, en tiempo real (dato que sale del equipo)
tonelaje_acumulado_anterior_carga = float(capacidad_en_peso_equipo_carguio * (rounded_prediction - 1))

#En producccion, este valor debera ser reemplazado por el tonelaje que acumula el equipo durante la carga, en tiempo real (dato que sale del equipo)
tonelaje_acumulado_carga = float(capacidad_en_peso_equipo_carguio * (rounded_prediction))

# Variable que se completo de acuerdo al expertise, pero puedo cambiar en las operaciones
payload = 1.05

# Variable que se completo de acuerdo al expertise, pero puedo cambiar en las operaciones
capacidad_llenado_vol = 1.05 


#Condicional

if (capacidad_en_peso_equipo_carguio * factor_llenado_peso / densidad_inicial_poligono_creado_tn_m3) > (capacidad_en_volumen_equipo_carguio_m3 * factor_llenado_volumen):
    cargado_real_cuchara_ton = capacidad_en_volumen_equipo_carguio_m3 * densidad_inicial_poligono_creado_tn_m3 * factor_llenado_volumen
else:
    cargado_real_cuchara_ton = capacidad_en_peso_equipo_carguio * factor_llenado_peso


pases_depositados = tonelaje_acumulado_carga / cargado_real_cuchara_ton


if (capacidad_en_volumen_equipo_carguio_m3 == 0 ):
    calculo_promedio_densidad_ton_m3 = densidad_inicial_poligono_creado_tn_m3
else:
    calculo_promedio_densidad_ton_m3 = densidad_inicial_poligono_creado_tn_m3 * (tonelaje_acumulado_carga - tonelaje_acumulado_anterior_carga) / cargado_real_cuchara_ton 

densidad_material_pase_ton_m3 = np.mean([densidad_inicial_poligono_creado_tn_m3, calculo_promedio_densidad_ton_m3, tonelaje_acumulado_carga/(capacidad_en_volumen_equipo_carguio_m3 * pases_depositados)])



if ((capacidad_en_peso_equipo_acarreo * payload) / densidad_material_pase_ton_m3 >  capacidad_en_volumen_equipo_acarreo_m3 * capacidad_llenado_vol ):
    tonelaje_requerido_camion_ton = capacidad_en_volumen_equipo_acarreo_m3 * densidad_material_pase_ton_m3 * capacidad_llenado_vol
else:
    tonelaje_requerido_camion_ton = capacidad_en_peso_equipo_acarreo * payload


# Realizar la predicción utilizando el modelo cargado
numero_cucharadas_requeridos = round(tonelaje_requerido_camion_ton / cargado_real_cuchara_ton, 2)

# formula 
diferencia = numero_cucharadas_requeridos - pases_depositados 


# Aplicar la condición utilizando if-else
if tiene_camion_cuadrado is True:
    if diferencia > 0.8:
        necesita_un_pase_mas = True
    else:
        necesita_un_pase_mas = False
else:
    if diferencia > 0.2:
        necesita_un_pase_mas = True
    else:
        necesita_un_pase_mas = False

# Mostrar el resultado
print("¿Necesita un pase más?:", necesita_un_pase_mas)